In [1]:
! pip install -U spacy[transformers]
! pip install transformers[sentencepiece]

     |████████████████████████████████| 3.1 MB 14.0 MB/s 
     |████████████████████████████████| 61 kB 598 kB/s 
     |████████████████████████████████| 596 kB 85.9 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 3.3 MB 68.7 MB/s 
     |████████████████████████████████| 1.2 MB 75.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:

! pip install matplotlib

In [8]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.1 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=6e6982348838ca8a31589029c40c09db83d57ea791df987d9b9482f92b8c15d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-w0lqitwd/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [12]:
# Load a language model and parse your document.
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

     |████████████████████████████████| 12.0 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
doc.ents

(Byonce, Jay Z.)

In [13]:
doc = nlp("If it is not used for hair, a round brush is an example of what")
print([(ent.text, ent.pos_) for ent in doc])

[('If', 'SCONJ'), ('it', 'PRON'), ('is', 'AUX'), ('not', 'PART'), ('used', 'VERB'), ('for', 'ADP'), ('hair', 'NOUN'), (',', 'PUNCT'), ('a', 'DET'), ('round', 'ADJ'), ('brush', 'NOUN'), ('is', 'AUX'), ('an', 'DET'), ('example', 'NOUN'), ('of', 'ADP'), ('what', 'PRON')]


In [15]:
from typing import List

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import networkx as nx
import torch
from torch import nn
from torch import nn, optim, Tensor
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from transformers import AlbertTokenizer, AlbertForMaskedLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import os
os.chdir('/content/drive/MyDrive/HES/NLP Final Project')

In [17]:
!ls

 cnet_embedding_data		    'Models - SquaD.docx'
 cnet_preprocess.ipynb		     question_answering.ipynb
'Copy of question_answering.ipynb'  'Stage 2 submission.gdoc'
 data				     tokenized_datasets_batched.pickle
 input.txt			     utils_numberbatch.py
 kgconstructor.py		     utils_numberbatch.py.ipynb
 kg_preprocess.ipynb


In [19]:
def brekdown(x):
    return x.split('\t')

In [22]:
columns = ['type', 'source', 'target', 'weight']
cnet_df = pd.DataFrame(columns=columns)
cnet_raw_df = pd.read_csv("/content/drive/MyDrive/HES/NLP Final Project/data/conceptnet.en.csv", header=None )
sample_df = cnet_raw_df.sample(1000).reset_index()
df = cnet_raw_df
m = len(df)
cnet_list=df[0].apply(brekdown)
columns = ['type', 'source', 'target', 'weight']
cnet_df = pd.DataFrame(list(cnet_list), columns=columns)

cnet_df.type = pd.Categorical(cnet_df.type)
cnet_df['code'] = cnet_df.type.cat.codes
cnet_df.to_csv('data/cnet_en_encoded.csv')

In [23]:
G = nx.from_pandas_edgelist(cnet_df, source='source', target='target', edge_attr=['weight', 'code'], create_using=nx.MultiDiGraph())

In [24]:
import pickle
with open("cnet_embedding_data/conceptnet_ih.graph", 'wb') as fr:
    pickle.dump(G, fr)

In [25]:
cnet_df.head()

,type,source,target,weight,code
0,antonym,ab_extra,ab_intra,1.0,0
1,antonym,ab_intra,ab_extra,1.0,0
2,antonym,abactinal,actinal,1.0,0
3,antonym,abandon,acquire,1.0,0
4,antonym,abandon,arrogate,1.0,0
